In [27]:
from datetime import date
from glob import glob

import pandas as pd

In [2]:
!ls -d machine_studies/*/

machine_studies/211125_all_agents_000_002/
machine_studies/211125_polished_donkey_22_66/
machine_studies/211202_all_agents_000_002/
machine_studies/211202_all_agents_000_002_solenoid_adjusted/
machine_studies/211202_polar_lake_000_069/
machine_studies/211202_polished_donkey_003_021/
machine_studies/211202_polished_donkey_067_099/
machine_studies/211209_all_agents_000_002_drift/
machine_studies/211209_all_three_drift/
machine_studies/211209_hannes_vs_agent_01/
machine_studies/211209_still_deluge_000_050/
machine_studies/211209_still_deluge_050_100/
machine_studies/211209_willi_vs_agent_01/
machine_studies/deluge_100_150/
machine_studies/deluge_150_200/
machine_studies/deluge_200_250/
machine_studies/donkey_100_150/
machine_studies/donkey_150_200/
machine_studies/donkey_200_250/
machine_studies/lake_100_150/
machine_studies/lake_150_200/
machine_studies/lake_200_250/
machine_studies/lake_70_100/
machine_studies/longrun/
machine_studies/shortrun/
machine_studies/snapshots/


## Over Problems Experiments

In [3]:
# Polished Donkey
filepaths = sorted(glob("machine_studies/*donkey*/*.pkl") + glob("machine_studies/211125_all_agents_000_002/*donkey*.pkl"))
dfdonkey = pd.concat([pd.read_pickle(path) for path in filepaths])

In [4]:
filepaths = sorted(glob("machine_studies/*lake*/*.pkl"))
dfpolar = pd.concat([pd.read_pickle(path) for path in filepaths])

In [5]:
filepaths = sorted(glob("machine_studies/*deluge*/*.pkl"))
dfstill = pd.concat([pd.read_pickle(path) for path in filepaths])

In [16]:
df = pd.concat([dfdonkey,dfpolar,dfstill])

In [17]:
df.drop("beam_image", axis=1, inplace=True)

In [18]:
df.to_pickle("machine_studies/summary_agentsoverproblems.pkl")

In [19]:
del dfdonkey
del dfpolar
del dfstill

## Drift Experiments

In [21]:
directories = sorted(glob("machine_studies/*_all_agents*"))
directories

['machine_studies/211125_all_agents_000_002',
 'machine_studies/211202_all_agents_000_002',
 'machine_studies/211202_all_agents_000_002_solenoid_adjusted',
 'machine_studies/211209_all_agents_000_002_drift']

In [22]:
comments = [None, None, "solenoid adjusted", None]
assert len(comments) == len(directories)

In [28]:
dates = [
    date(year=2021, month=11, day=25),
    date(year=2021, month=12, day=2),
    date(year=2021, month=12, day=2),
    date(year=2021, month=12, day=9)
]
assert len(dates) == len(directories)

In [29]:
dfs = []
for directory, comment, rundate in zip(directories, comments, dates):
    filepaths = glob(f"{directory}/*.pkl")
    df = pd.concat([pd.read_pickle(path) for path in filepaths])
    df["comment"] = comment
    df["date"] = rundate
    dfs.append(df)
df = pd.concat(dfs)

In [33]:
df.drop("beam_image", axis=1, inplace=True)

In [34]:
df.to_pickle("machine_studies/summary_driftexperiments.pkl")

## Agent vs. Operator

## Long Run